In [ ]:
#| default_exp markdown.obsidian.personal.notation.parse

# markdown.obsidian.personal.notation.parse
> For parsing notation notes

In [ ]:
#| export
from os import PathLike
import re
from typing import Optional, Union

from trouver.markdown.markdown.file import MarkdownFile, MarkdownLineEnum 
from trouver.markdown.obsidian.links import link_ranges_in_text, ObsidianLink, MARKDOWNLINK_PATTERN, MARKDOWNLINK_CAPTURE_PATTERN, WIKILINK_PATTERN
from trouver.markdown.obsidian.vault import VaultNote
from trouver.markdown.obsidian.personal.note_type import assert_note_is_of_type, PersonalNoteTypeEnum

In [ ]:
from fastcore.test import *
from trouver.helper.tests import _test_directory

# Parse notation note

For the purposes of `trouver`, a notation note most usually starts in the format `<Notation> [[<link_to_note>|denotes]] <explanation of what that notation denotes/is defined as> <optional paragraphs discussing aspects about the notation>`. For example, "$\deg D$ [[note_link|denotes]] the degree of the divisor $D$" would be an example of such a note.

If the word `denotes` is not given in a link to a note, then the note to which the first link points is considered the main note of the notation note. Alternatively, a notation note might also sometimes have a comment citing the source of the notation instead of links.  Nevertheless, it is preferred that `denotes` is given in a link.
 
A notation note may have YAML frontmatter meta as well.


In [ ]:
#| export
def _main_of_notation_from_text(
        file_text: str # Text of notation note
        ) -> Union[str, VaultNote, None]: # The name main information note that `notation_note` comes from. Returns `None` if `notation_note` does not come from such a note.
    """Return the name of the note from which the notation comes from.

    Helper function for `parse_notation_string`.
    """
    if '%%' in file_text and 'main: ' in file_text:
        return None

    # First, get rid of the notation, i.e. the first latex math mode string
    match = re.search(fr'\$.+?\$\s+', file_text) 
    if match is None:
        raise ValueError(
            'There seems to be a formatting error in a notation note'
            ' and the notation has not been identified. The following is the'
            f' content of the notation note:\n{file_text}')
    start, end = match.span()
    file_text = file_text[end:]
    
    link_locations = link_ranges_in_text(file_text)
    if not link_locations:
        return None
    start, end = link_locations[0]
    link_str = file_text[start:end]
    link_parse = ObsidianLink.from_text(link_str)
    main_note_name = link_parse.file_name
    return main_note_name

In [ ]:
#| hide
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
test_eq(_main_of_notation_from_text(notation_note.text()), 'spectrum_of_a_ring')

In [ ]:
#| hide
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_O_X_this_file_has_no_links')
main_note =  _main_of_notation_from_text(notation_note.text())
assert main_note is None

In [ ]:
#| hide
# Test the case where the notation string contains a [[]]
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_k_t_formal_power_series_ring') 
_main_of_notation_from_text(notation_note.text())


'some_note'

In [ ]:
#| hide
# TODO: test error being raised

In [ ]:
#| export
def _divide_bulleted_list_mf_at_end(
        mf: MarkdownFile
        ) -> tuple[MarkdownFile, list[tuple[str, str]]]: # The first MarkdownFile contains the main content. The second MarkdonwFile contains the bulleted list at the end; if no such bulleted list exists, then this is None.
    """Return a `MarkdownFile` consisting of just the main content
    of a notation note along with the list of tuples capturing the
    information of the trailing bulleted list of links
    for notations used in the notation note.

    Assumes that the bulleted list is formatted correctly
    (i.e. each line is of the form `- [<notation>](<link>)`)

    Helper function for `parse_notation_note`.
    """
    main_parts = mf.parts.copy()
    trailing_parts = []
    for part in reversed(mf.parts):
        if part['type'] == MarkdownLineEnum.BLANK_LINE:
            main_parts.pop()
            continue
        if not _part_is_unordered_list_and_is_of_markdownstyle_link(part):
            break
        last_part = main_parts.pop() # Should be the same as `part`
        trailing_parts.insert(0, last_part)
    
    bulleted_list = [
        _notat_str_and_linked_notat_note_name_from_bullet_point_part(part)
        for part in trailing_parts]
    return MarkdownFile(main_parts), bulleted_list
    

def _part_is_unordered_list_and_is_of_markdownstyle_link(
        part: dict[str, Union[str, MarkdownLineEnum]]
        ) -> bool:
    """
    
    Helper function for `_divide_bulleted_list_mf_at_end`
    """
    if part['type'] != MarkdownLineEnum.UNORDERED_LIST:
        return False
    if not part['line'].startswith('- '):
        return False
    if not re.match(MARKDOWNLINK_PATTERN, part['line'][2:]):
        return False
    return True
    

def _notat_str_and_linked_notat_note_name_from_bullet_point_part(
        part: dict[str, Union[str, MarkdownLineEnum]]
        ) -> tuple[str, str]:
    match = re.match(MARKDOWNLINK_CAPTURE_PATTERN, part['line'][2:])
    return match[1], match[2]

In [ ]:
test_eq(['hi'], ['hi'])

In [ ]:
#| hide
notation_note_str = r'''$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.
- [$L$](notation_L_some_field_extension)
- [$K$](notation_K_some_base_field)'''
mf = MarkdownFile.from_string(notation_note_str)
main_content, bulleted_list = _divide_bulleted_list_mf_at_end(mf)
test_eq(str(main_content), r'$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.')
test_eq(bulleted_list, [('$L$', 'notation_L_some_field_extension'), ('$K$', 'notation_K_some_base_field')])

# This time, add more blank lines
notation_note_str = r'''$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.

- [$L$](notation_L_some_field_extension)

- [$K$](notation_K_some_base_field)
'''
mf = MarkdownFile.from_string(notation_note_str)
main_content, bulleted_list = _divide_bulleted_list_mf_at_end(mf)
test_eq(str(main_content), r'$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.')
test_eq(bulleted_list, [('$L$', 'notation_L_some_field_extension'), ('$K$', 'notation_K_some_base_field')])

# No bulleted list means the second output is `None`:

notation_note_str = r'''$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.

'''
mf = MarkdownFile.from_string(notation_note_str)
main_content, bulleted_list = _divide_bulleted_list_mf_at_end(mf)
test_eq(str(main_content), r'$\operatorname{Gal}(L/K)$ [[linky|denotes]] blah blah blah.')
test_eq(bulleted_list, [])



In [ ]:
#| export
def parse_notation_note(
        notation_note: Union[str, VaultNote],
        vault: Optional[PathLike] = None # The vault If `None`, then uses `notation_note.vault`
        ) -> tuple[Union[dict, None], Union[str, None], str, MarkdownFile,
                   list[tuple[str, str]]]:
    """Parse information from the notation note.

    **Returns**

    - tuple[Union[dict, None], str, ObsidianLink, MarkdownFile, list[tuple[str, str]]]
        - The first entry is the YAML frontmatter meta, if available.
        - The second entry is the notation string
        - The third entry is the name of the "main note" of the notation note. This is usual
          the linked note in the link `[[<linked_note>|denotes]]`. If no such main note
          exists, then this is `None`.
        - The fourth entry is the MarkdownFile consisting of the "main" content of the note,
          which excludes the information given by all of the other entries.
        - The fifth entry is a list of tuples of two `str`'s representing a
          bulleted list of notation notes to which `notation_note` links to.
          Each tuple is of the form `(latex_str, notation_note_name)` and
          the corresponding entry in the bulleted list is of the form
          `- [<latex_str>](<notation_note_name)`.

    **Raises**

    - UserWarning
        - If the (non-YAML frontmatter meta) contents of the note do not start
        inn the form `<Notation> [[<link_to_note>|denotes]]`; the name of the
        notation note is included in the warning message.
    - ValueError
        - If the notation note is not formatted correctly by starting
        with the notation with dollar signs `$`.
    - AssertionError
        - If `notation_note` is not determined to be a notation note.
    """
    if isinstance(notation_note, str):
        notation_note = VaultNote(vault, name=notation_note)
    if not vault:
        vault = notation_note.vault
    assert_note_is_of_type(notation_note, PersonalNoteTypeEnum.NOTATION_NOTE)

    mf = MarkdownFile.from_vault_note(notation_note)
    try:
        metadata = mf.metadata()
    except ValueError as e:
        print(f'The following notation note has some kind of formatting error in its YAML frontmatter metadata: {notation_note.name}')
        raise(e)
    mf_without_metadata = MarkdownFile(
        [part for part in mf.parts if part['type'] != MarkdownLineEnum.META])

    file_text = str(mf_without_metadata)

    main_mf, linked_notations_list = _divide_bulleted_list_mf_at_end(mf_without_metadata)
    _remove_the_notation_str_and_denotes_in_main_mf(main_mf, notation_note)

    return (metadata, _get_notation_string(file_text, notation_note),
            _main_of_notation_from_text(file_text), main_mf,
            linked_notations_list)


def _get_notation_string(
        file_text: str,
        notation_note: VaultNote
        ) -> str:
    """Return the notation string from the text of the notation note..

    Assumes that the notation string exists and is well formatted.

    Helper function for `parse_notation_note`.
    """
    try:
        return re.search(r'\$.+?\$', file_text).group()
    except AttributeError as e:
        raise ValueError(
            'There seems to be a formatting error in a notation note'
            ' and the notation has not been identified. The following is the'
            f' name of the notation note: {notation_note.name}')
    

def _remove_the_notation_str_and_denotes_in_main_mf(
        main_mf: MarkdownFile,
        notation_note: VaultNote):
    """Remove the text `<notation> denotes ` which starts the
    notation description.

    Helper function of `parse_notation_note`.
    """
    for part in main_mf.parts:
        if part['type'] == MarkdownLineEnum.BLANK_LINE:
            continue
        match = re.match(fr'^\$.+?\$ ({WIKILINK_PATTERN}|denotes)\s*', part['line']) 
        if match is None:
            raise ValueError(
                'There seems to be a formatting error in a notation note'
                ' and the notation has not been identified. The following is the'
                f' name of the notation note: {notation_note.name}')
        else:
            start, end = match.span()
            part['line'] = part['line'][end:]
            break
    

`parse_notation_note` gets information about the notation note. Note that the `MarkdownFile` object `main_mf` that has the main content/description of the notation does not start with the pharse of the form `<notation> [[<link>|denotes]] `.

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
metadata, notation_str, main_of_notation, main_mf, linked_notat_notes = parse_notation_note(notation_note, vault)

test_eq(metadata, {'detect_regex': [], 'latex_in_original': ['\\operatorname{Spec} A']})
test_eq(notation_str, '$\\operatorname{Spec} A$')
test_eq(main_of_notation, 'spectrum_of_a_ring')
test_eq(str(main_mf), 'the spectrum of the ring $A$.')
test_eq(linked_notat_notes, []) # There is not a bulleted list at the end, so the last output is `None`.

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='poonen_curves_notation_zeta_X_s_zeta_function_of_variety')
metadata, notation_str, main_of_notation, main_mf, linked_notat_notes = parse_notation_note(notation_note, vault)

test_eq(metadata, None)
test_eq(notation_str, r'$\zeta_{X}(s)$')
test_eq(main_of_notation, 'poonen_curves_3.4.1 DEFINITION')
test_eq(str(main_mf), 'the zeta function of the [[poonen_curves_1.0.2 DEFINITION|variety]] $X$ over $\\mathbb{F}_q$.\n\nIt is defined as\n\n$$\\zeta_X(s) = Z_X(q^{-s}).$$\n\nA priori, it is a formal series, but in fact [[poonen_curves_ 3.6_page_56|it converges]] for $\\operatorname{Re} s > \\dim X$.')
test_eq(linked_notat_notes, [('$Z_X$', 'poonen_curves_notation_Z_X_T')]) # There is a bulleted list at the end, so the last output is `None`.

In [ ]:
#| hide
# Test the case where the note has no main note; in particular, it starts with `<notation> denotes ` with no link!`

vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_O_X_this_file_has_no_links') 
metadata, notation_str, main_of_notation, main_mf, linked_notat_notes = parse_notation_note(notation_note, vault)
assert main_note is None


In [ ]:
#| hide
# Test the case where the note not blank lines after the `denotes`. 

vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_B_R') 
metadata, notation_str, main_of_notation, main_mf, linked_notat_notes = parse_notation_note(notation_note, vault)
assert main_note is None


In [ ]:
#| hide
# Test the case where the notation string contains a [[]]
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_k_t_formal_power_series_ring') 
metadata, notation_str, main_of_notation, main_mf, linked_notat_notes = parse_notation_note(notation_note, vault)
test_eq(notation_str, '$k[[t]]$')
test_eq(main_of_notation, 'some_note')
# assert main_note is None

In [ ]:
#| export
def notation_in_note(
        notation_note: Union[str, VaultNote],
        vault: Optional[PathLike] = None 
        ) -> str:
    """Return the name of the note from which the notation comes from.
    
    **Parameters**

    - `notation_note` - Union[str, VaultNote]
        - Either
            
            - The name of the notation note or
            - The `VaultNote` object of the notation note. 
            
        The note name is expected to be unique
        inside the vault specified by `vault`. 
        This is expected to contain `'notation'` as a substring. 
        Usually, this is expected to be formatted in one of
        the following forms:
            - `'<reference_name>_notation_<rest_of_note_name>'`
            - `'notation.<rest_of_note_name>'
    - `vault` - Pathlike or `None`
        - Defaults to `None`
        
    **Returns**

    - str
        - The notation in LaTeX, including the dollar signs `$`.

    **Raises**

    """
    _, notation_in_note, _, _, _ = parse_notation_note(notation_note, vault)
    return notation_in_note

`notation_in_note` identifies the notation LaTeX str that a notation note presents. Its output starts and ends with dollar signs.

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
notation = notation_in_note(notation_note)
assert notation == r'$\operatorname{Spec} A$'

In [ ]:
#| export
def main_of_notation(
        notation_note: VaultNote, # The VaultNote object representing the notation note.
        as_note: bool = False # If `False`, then returns the name of the note, and returns a VaultNote object with the same vault as `notation_note` otherwise. The vault used to get the `VaultNote` is the vault of `notation_note`.
        ) -> Union[str, VaultNote, None]: # The (name of the) main information note that `notation_note` comes from. Returns `None` if `notation_note` does not come from such a note.
    """Return the name of the note from which the notation comes from.
            
    **Raises**

    - ValueError
        - If the notation note is not formatted correctly by starting
        with the notation with dollar signs `$`.
    """
    vault = notation_note.vault
    _, _, main_note_name, _, _ = parse_notation_note(notation_note, vault)
    if main_note_name is None:
        return None
    if as_note:
        return VaultNote(notation_note.vault, name=main_note_name)
    else:
        return main_note_name

We can identify the "main note" of a notation note with the `main_of_notation` method:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
test_eq(main_of_notation(notation_note), 'spectrum_of_a_ring')

We can also return this main note as a `VaultNote` object:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A')
main_note =  main_of_notation(notation_note, as_note=True)
assert isinstance(main_note, VaultNote)
test_eq(main_note.name, 'spectrum_of_a_ring')

If the notation note has no links, then `main_of_notation` returns `None`:

In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_O_X_this_file_has_no_links')
main_note =  main_of_notation(notation_note)
assert main_note is None

main_note = main_of_notation(notation_note, as_note=True)
assert main_note is None